In [1]:
import pandas as pd

In [2]:
stops = pd.read_csv("../../data/S_VISTA_1220_SA1.csv")
stops = stops[stops["wdstopwgt_sa3"].notna()]

C:\Users\dlaa0001\AppData\Local\Temp\ipykernel_24296\1259530509.py:1: DtypeWarning: Columns (46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  stops = pd.read_csv("../../data/S_VISTA_1220_SA1.csv")


In [3]:
stops.columns

Index(['stopid', 'persid', 'hhid', 'stopno', 'starthour', 'startime',
       'arrhour', 'arrtime', 'duration', 'dephour', 'deptime', 'travtime',
       'vistadist', 'origplace1', 'origplace2', 'origpurp1', 'origpurp2',
       'destplace1', 'destplace2', 'destpurp1', 'destpurp2', 'origsa1',
       'origsa2', 'origsa2_name', 'origsa3', 'origsa3_name', 'origsa4',
       'origsa4_name', 'origlga', 'destsa1', 'destsa2', 'destsa2_name',
       'destsa3', 'destsa3_name', 'destsa4', 'destsa4_name', 'destlga',
       'fullmode', 'mainmode', 'person1', 'person2', 'person3', 'person4',
       'person5', 'person6', 'person7', 'person8', 'person9', 'person10',
       'person11', 'wdstopwgt_sa3', 'westopwgt_sa3', 'wdstopwgt_lga',
       'westopwgt_lga'],
      dtype='object')

In [4]:
chosen_cols = ["persid", "stopid", "hhid", "stopno", "origplace1", "destplace1", "origpurp1", "destpurp1", "startime", "arrtime", "duration", "travtime"]
sub_stops = stops[chosen_cols]
sub_stops

,persid,stopid,hhid,stopno,origplace1,destplace1,origpurp1,destpurp1,startime,arrtime,duration,travtime
0,Y12H0000101P01,Y12H0000101P01S01,Y12H0000101,1,Accommodation,Workplace,At Home,Work Related,525,545,505.0,20
1,Y12H0000101P01,Y12H0000101P01S02,Y12H0000101,2,Workplace,Accommodation,Work Related,At or Go Home,1050,1075,NaN,25
2,Y12H0000101P02,Y12H0000101P02S01,Y12H0000101,1,Accommodation,Place of Education,At Home,Pick-up or Drop-off Someone,500,520,20.0,20
3,Y12H0000101P02,Y12H0000101P02S02,Y12H0000101,2,Place of Education,Shops,Pick-up or Drop-off Someone,Buy Something,540,550,50.0,10
4,Y12H0000101P02,Y12H0000101P02S03,Y12H0000101,3,Shops,Accommodation,Buy Something,At or Go Home,600,610,280.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...
247820,Y20H4001029P02,Y20H4001029P02S01,Y20H4001029,1,Accommodation,Place of Personal Business,At Home,Personal Business,600,620,5.0,20
247821,Y20H4001029P02,Y20H4001029P02S02,Y20H4001029,2,Place of Personal Business,Accommodation,Personal Business,At or Go Home,625,640,NaN,15
247822,Y20H4001039P01,Y20H4001039P01S01,Y20H4001039,1,Accommodation,Place of Personal Business,At Home,Personal Business,540,550,40.0,10
247823,Y20H4001039P01,Y20H4001039P01S02,Y20H4001039,2,Place of Personal Business,Place of Personal Business,Personal Business,Personal Business,590,650,70.0,60


In [5]:
have_first_stop = sub_stops.groupby("persid")["stopno"].apply(lambda x: list(x)).apply(lambda x: 1 in x)[lambda x: x]
sub_stops = sub_stops[sub_stops["persid"].isin(have_first_stop.index)]

In [6]:
# Group by persid and get first/last locations
first_las_loc_check = (
    sub_stops
    .sort_values("stopno")
    .groupby("persid")
    .agg({
        "origplace1": "first",
        "destplace1": "last"
    })
    .reset_index()
)
home_bound_pp = first_las_loc_check[(first_las_loc_check["origplace1"] == "Accommodation") & (first_las_loc_check["destplace1"] == "Accommodation")]
assert home_bound_pp["origplace1"].nunique() == home_bound_pp["destplace1"].nunique() == 1

In [7]:
final_stops = sub_stops[sub_stops["persid"].isin(home_bound_pp["persid"])]
final_stops["destpurp1"].value_counts()

destpurp1
At or Go Home                   61679
Change Mode                     31644
Work Related                    24857
Buy Something                   13953
Pick-up or Drop-off Someone     12463
Social                          12272
Personal Business                7852
Education                        7758
Recreational                     7445
Accompany Someone                5664
Pick-up or Deliver Something     2183
Other Purpose                     396
Not Stated                          4
Unknown Purpose                     1
Name: count, dtype: int64

In [8]:
purpose_groups = {
    # Home & Transit
    "At or Go Home": "Home",
    "Change Mode": "Transit",

    # Work & Education
    "Work Related": "Work",
    "Education": "Education",

    # Shopping
    "Buy Something": "Shopping",
    "Personal Business": "Shopping",
    "Pick-up or Deliver Something": "Shopping",

    # Social
    "Social": "Social",
    "Recreational": "Social",

    # Accompanying
    "Pick-up or Drop-off Someone": "Accompanying",
    "Accompany Someone": "Accompanying",

    # Other
    "Other Purpose": "Other",
    "Not Stated": "Other",
    "Unknown Purpose": "Other"
}
final_stops["purpose"] = final_stops["destpurp1"].map(purpose_groups)
final_stops["purpose"].value_counts()

C:\Users\dlaa0001\AppData\Local\Temp\ipykernel_24296\3519503332.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_stops["purpose"] = final_stops["destpurp1"].map(purpose_groups)


purpose
Home            61679
Transit         31644
Work            24857
Shopping        23988
Social          19717
Accompanying    18127
Education        7758
Other             401
Name: count, dtype: int64

In [9]:
# Get the last stop for each person
last_stops = final_stops.groupby("persid")["stopno"].max().reset_index()
last_stop_check = final_stops.merge(last_stops, on=["persid", "stopno"])
# Assert that duration is NaN for all last stops
# assert last_stop_check["duration"].isna().all(), "Some last stops have non-NaN duration values"
to_rm_pp = last_stop_check[(last_stop_check["stopno"] == 1) | (last_stop_check["purpose"] != "Home")]["persid"]
final_stops = final_stops[~final_stops["persid"].isin(to_rm_pp)]
last_stop_check = last_stop_check[~last_stop_check["persid"].isin(to_rm_pp)]
final_stops["purpose"].value_counts()

purpose
Home            61353
Transit         31170
Work            24658
Shopping        23133
Social          19238
Accompanying    17959
Education        7669
Other             386
Name: count, dtype: int64

In [10]:
import numpy as np
# Set duration to NaN for last stops
if final_stops.index.name != "stopid":
    final_stops = final_stops.set_index("stopid")
final_stops.loc[last_stop_check["stopid"], "duration"] = np.nan

# 1800 represents end of day (30 hours * 60 minutes)
final_stops["duration"] = final_stops["duration"].fillna(1800 - final_stops["arrtime"])

# Assert no negative durations
assert (final_stops["duration"] >= 0).all(), "Found negative duration values"

In [11]:
final_stops["total_duration"] = final_stops["duration"] + final_stops["travtime"]
final_stops

,persid,hhid,stopno,origplace1,destplace1,origpurp1,destpurp1,startime,arrtime,duration,travtime,purpose,total_duration
stopid,,,,,,,,,,,,,
Y12H0000101P01S01,Y12H0000101P01,Y12H0000101,1,Accommodation,Workplace,At Home,Work Related,525,545,505.0,20,Work,525.0
Y12H0000101P01S02,Y12H0000101P01,Y12H0000101,2,Workplace,Accommodation,Work Related,At or Go Home,1050,1075,725.0,25,Home,750.0
Y12H0000101P02S01,Y12H0000101P02,Y12H0000101,1,Accommodation,Place of Education,At Home,Pick-up or Drop-off Someone,500,520,20.0,20,Accompanying,40.0
Y12H0000101P02S02,Y12H0000101P02,Y12H0000101,2,Place of Education,Shops,Pick-up or Drop-off Someone,Buy Something,540,550,50.0,10,Shopping,60.0
Y12H0000101P02S03,Y12H0000101P02,Y12H0000101,3,Shops,Accommodation,Buy Something,At or Go Home,600,610,280.0,10,Home,290.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y20H4001029P02S01,Y20H4001029P02,Y20H4001029,1,Accommodation,Place of Personal Business,At Home,Personal Business,600,620,5.0,20,Shopping,25.0
Y20H4001029P02S02,Y20H4001029P02,Y20H4001029,2,Place of Personal Business,Accommodation,Personal Business,At or Go Home,625,640,1160.0,15,Home,1175.0
Y20H4001039P01S01,Y20H4001039P01,Y20H4001039,1,Accommodation,Place of Personal Business,At Home,Personal Business,540,550,40.0,10,Shopping,50.0


In [12]:
filter_final_stops = final_stops[["persid", "hhid", "stopno", "purpose", "startime", "total_duration"]]
filter_final_stops

,persid,hhid,stopno,purpose,startime,total_duration
stopid,,,,,,
Y12H0000101P01S01,Y12H0000101P01,Y12H0000101,1,Work,525,525.0
Y12H0000101P01S02,Y12H0000101P01,Y12H0000101,2,Home,1050,750.0
Y12H0000101P02S01,Y12H0000101P02,Y12H0000101,1,Accompanying,500,40.0
Y12H0000101P02S02,Y12H0000101P02,Y12H0000101,2,Shopping,540,60.0
Y12H0000101P02S03,Y12H0000101P02,Y12H0000101,3,Home,600,290.0
...,...,...,...,...,...,...
Y20H4001029P02S01,Y20H4001029P02,Y20H4001029,1,Shopping,600,25.0
Y20H4001029P02S02,Y20H4001029P02,Y20H4001029,2,Home,625,1175.0
Y20H4001039P01S01,Y20H4001039P01,Y20H4001039,1,Shopping,540,50.0


In [13]:
# Get first stop for each person to calculate initial duration
first_stops = filter_final_stops[filter_final_stops["stopno"] == 1].copy()

# Create stopno 0 entries
zero_stops = first_stops.copy()
zero_stops["stopno"] = 0
zero_stops["purpose"] = "Home"
zero_stops["startime"] = 0
zero_stops["total_duration"] = first_stops["startime"]

# Combine with original stops and sort
filter_final_stops = pd.concat([zero_stops, filter_final_stops])
filter_final_stops = filter_final_stops.sort_values(["persid", "stopno"]).reset_index(drop=True)
filter_final_stops

,persid,hhid,stopno,purpose,startime,total_duration
0,Y12H0000101P01,Y12H0000101,0,Home,0,525.0
1,Y12H0000101P01,Y12H0000101,1,Work,525,525.0
2,Y12H0000101P01,Y12H0000101,2,Home,1050,750.0
3,Y12H0000101P02,Y12H0000101,0,Home,0,500.0
4,Y12H0000101P02,Y12H0000101,1,Accompanying,500,40.0
...,...,...,...,...,...,...
229014,Y20H4001029P02,Y20H4001029,2,Home,625,1175.0
229015,Y20H4001039P01,Y20H4001039,0,Home,0,540.0
229016,Y20H4001039P01,Y20H4001039,1,Shopping,540,50.0
229017,Y20H4001039P01,Y20H4001039,2,Shopping,590,130.0


In [14]:
persons_atts = pd.read_csv("../../data/P_VISTA_1220_SA1.csv")
persons_atts.columns

Index(['persid', 'hhid', 'persno', 'numstops', 'monthofbirth', 'yearofbirth',
       'age', 'sex', 'relationship', 'persinc', 'carlicence', 'mbikelicence',
       'otherlicence', 'nolicence', 'fulltimework', 'parttimework',
       'casualwork', 'anywork', 'studying', 'activities', 'mainact',
       'worktype', 'emptype', 'anzsco1', 'anzsco2', 'anzsic1', 'anzsic2',
       'startplace', 'additionaltravel', 'cycledwork', 'cycledshopping',
       'cycledexercise', 'cycledother', 'nocycled', 'wdperswgt_sa3',
       'weperswgt_sa3', 'wdperswgt_lga', 'weperswgt_lga'],
      dtype='object')

In [15]:
chosen_persons_cols = ["persid", "hhid", "age", "sex", "persinc", "nolicence", 'fulltimework', 'parttimework', 'casualwork', 'anywork', 'studying', "worktype"]
sub_persons = persons_atts[chosen_persons_cols]
sub_persons = sub_persons[sub_persons["persid"].isin(filter_final_stops["persid"])]
sub_persons

,persid,hhid,age,sex,persinc,nolicence,fulltimework,parttimework,casualwork,anywork,studying,worktype
0,Y12H0000101P01,Y12H0000101,50,M,$1000-1249 p.w.,Some Licence,Y,N,N,Y,No Study,Fixed Hours
1,Y12H0000101P02,Y12H0000101,43,F,$1-199 p.w.,Some Licence,N,N,N,N,No Study,Not in Work Force
2,Y12H0000101P03,Y12H0000101,11,F,Zero Income,No Licence,N,N,N,N,Primary,Not in Work Force
3,Y12H0000101P04,Y12H0000101,6,F,Zero Income,No Licence,N,N,N,N,Primary,Not in Work Force
5,Y12H0000102P02,Y12H0000102,56,M,$300-399 p.w.,Some Licence,N,N,N,N,No Study,Not in Work Force
...,...,...,...,...,...,...,...,...,...,...,...,...
78964,Y20H4001009P01,Y20H4001009,59,F,$400-599 p.w.,Some Licence,N,Y,N,Y,No Study,Fixed Hours
78965,Y20H4001009P02,Y20H4001009,57,M,$1000-1249 p.w.,Some Licence,Y,N,N,Y,No Study,Fixed Hours
78966,Y20H4001009P03,Y20H4001009,25,F,$1250-1499 p.w.,Some Licence,Y,N,N,Y,No Study,Fixed Hours
78975,Y20H4001029P02,Y20H4001029,70,F,$300-399 p.w.,Some Licence,N,N,N,N,No Study,Not in Work Force


In [16]:
filter_final_stops.to_csv("../../data/activities_homebound_wd.csv", index=False)
sub_persons.to_csv("../../data/persons_homebound_wd.csv", index=False)

In [17]:
filter_final_stops["persid"].value_counts()

persid
Y19H3612401P01    29
Y15H1570503P01    25
Y16H1990323P02    25
Y19H3530411P01    24
Y19H3840302P02    24
                  ..
Y20H4000236P01     3
Y20H4000229P02     3
Y20H4000229P01     3
Y20H4000424P02     3
Y20H4000424P01     3
Name: count, Length: 43453, dtype: int64

In [18]:
small_set_persons = sub_persons.sample(n=50)
small_set_stops = filter_final_stops[filter_final_stops["persid"].isin(small_set_persons["persid"])]
small_set_stops.to_csv("../../data/small_activities_homebound_wd.csv", index=False)
small_set_persons.to_csv("../../data/small_persons_homebound_wd.csv", index=False)
